In [1]:
# import dependencies
import pandas as pd
import os

In [2]:
# assign file path to variable
pymo_file = os.path.join('purchase_data2.json')

In [3]:
# create dataframe with purchase data
pymo_raw = pd.read_json(pymo_file)
pymo_raw.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,20,Male,93,Apocalyptic Battlescythe,4.49,Iloni35
1,21,Male,12,Dawne,3.36,Aidaira26
2,17,Male,5,Putrid Fan,2.63,Irim47
3,17,Male,123,Twilight's Carver,2.55,Irith83
4,22,Male,154,Feral Katana,4.11,Philodil43


In [4]:
player_count = pymo_raw['SN'].nunique()

# create summary DataFrame with purchase data
play_count = pd.DataFrame({'Player Count': player_count} , index=[0])
play_count

,Player Count
0,74


In [5]:
unique_items = len(pymo_raw['Item Name'].unique())

In [6]:
avg_pur_price = round(pymo_raw['Price'].mean(),2)

In [7]:
total_purchases = len(pymo_raw)

In [8]:
total_revenue = round(pymo_raw['Price'].sum(),2)

In [9]:
# Purchasing Anaysis (Total)
# create dictionary to hold calculated data points
purchase_data = {
    'Unique Items': unique_items,
    'Avg Price': avg_pur_price,
    'Total Purchases': total_purchases,
    'Total Revenue': total_revenue
}

# create summary DataFrame with purchase data
purch_analysis = pd.DataFrame(purchase_data , index=[0])

# format columns with currency values
purch_analysis['Avg Price'] = purch_analysis['Avg Price'].map("${:.2f}".format)
purch_analysis['Total Revenue'] = purch_analysis['Total Revenue'].map("${:.2f}".format)

purch_analysis

,Avg Price,Total Purchases,Total Revenue,Unique Items
0,$2.92,78,$228.10,63


In [10]:
# Gender Demographics
# group raw data by gender
pymo_gdr = pymo_raw.groupby(['Gender'])

# get count of players by gender
pymo_gdr_count = pymo_gdr['SN'].nunique()

# calculate count by gender as a percentage of all players
pymo_gdr_pct = round(pymo_gdr_count / player_count, 4) * 100

# add count and percentage to summary DataFrame
gdr_demo = pd.DataFrame({
    'Player count': pymo_gdr_count,
    'Percentage of Players': pymo_gdr_pct
})
gdr_demo

,Percentage of Players,Player count
Gender,,
Female,17.57,13
Male,81.08,60
Other / Non-Disclosed,1.35,1


In [11]:
# Gender Purchase Analysis
# create DataFrame with count mean and sum of Price series by gender
purch_analysis_gdr = pymo_gdr['Price'].agg(['count', 'mean', 'sum'])

# change DataFrame column names
purch_analysis_gdr = purch_analysis_gdr.rename(columns={
    'count': 'Total Purchases',
    'mean': 'Avg Price',
    'sum': 'Total Revenue'
})

# add gender count to gender purchase analysis
purch_analysis_gdr['Player Count'] = pymo_gdr_count

# calculate normalized total
normalized_total_gdr = purch_analysis_gdr['Total Revenue'] / purch_analysis_gdr['Player Count']
purch_analysis_gdr['Total Revenue (Normalized)'] = normalized_total_gdr

# format columns with currency values
purch_analysis_gdr['Avg Price'] = purch_analysis_gdr['Avg Price'].map("${:.2f}".format)
purch_analysis_gdr['Total Revenue'] = purch_analysis_gdr['Total Revenue'].map("${:.2f}".format)
purch_analysis_gdr['Total Revenue (Normalized)'] = purch_analysis_gdr['Total Revenue (Normalized)'].map("${:.2f}".format)

# rearrange columns
purch_analysis_gdr = purch_analysis_gdr[[
    'Player Count', 'Total Purchases',
    'Avg Price', 'Total Revenue',
    'Total Revenue (Normalized)'
]]   

purch_analysis_gdr

,Player Count,Total Purchases,Avg Price,Total Revenue,Total Revenue (Normalized)
Gender,,,,,
Female,13,13,$3.18,$41.38,$3.18
Male,60,64,$2.88,$184.60,$3.08
Other / Non-Disclosed,1,1,$2.12,$2.12,$2.12


In [12]:
# Age Demographics
# find min and max age
min_age = pymo_raw['Age'].min()
max_age = pymo_raw['Age'].max()

print('Youngest Player: ' + str(min_age))
print('Oldest Player: ' + str(max_age))

# define bins limits for age groups
age_bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]

# define bin categories for age groups
age_cat = ['<10', '10-14', '15-19', '20-24',
           '25-29', '30-34', '35-39', '40+']

# assign categories and add to pymoli age dataframe
pymo_age_assign = pd.cut(pymo_raw['Age'], age_bins, labels=age_cat)

# add age groups to pymo_raw DataFrame
pymo_raw['Age Groups'] = pymo_age_assign

# group raw data by Age Groups
pymo_age = pymo_raw.groupby('Age Groups')

# get count of players by gender
pymo_age_count = pymo_age['SN'].nunique()

# calculate count by gender as a percentage of all players
pymo_age_pct = round(pymo_age_count / player_count, 4) * 100

# add count and percentage to summary DataFrame
age_demo = pd.DataFrame({
    'Player count': pymo_age_count,
    'Percentage of Players': pymo_age_pct
})
age_demo

Youngest Player: 7
Oldest Player: 40


,Percentage of Players,Player count
Age Groups,,
<10,6.76,5
10-14,4.05,3
15-19,14.86,11
20-24,45.95,34
25-29,10.81,8
30-34,8.11,6
35-39,8.11,6
40+,1.35,1


In [13]:
# Age Purcahase Analysis
# create DataFrame with count mean and sum of Price series by Age Group
purch_analysis_age = pymo_age['Price'].agg(['count', 'mean', 'sum'])

# change DataFrame column names
purch_analysis_age = purch_analysis_age.rename(columns={
    'count': 'Total Purchases',
    'mean': 'Avg Price',
    'sum': 'Total Revenue'
})

# add gender count to gender purchase analysis
purch_analysis_age['Player Count'] = pymo_age_count

# calculate normalized total
normalized_total_age = purch_analysis_age['Total Revenue'] / purch_analysis_age['Player Count']
purch_analysis_age['Total Revenue (Normalized)'] = normalized_total_age

# format columns with currency values
purch_analysis_age['Avg Price'] = purch_analysis_age['Avg Price'].map("${:.2f}".format)
purch_analysis_age['Total Revenue'] = purch_analysis_age['Total Revenue'].map("${:.2f}".format)
purch_analysis_age['Total Revenue (Normalized)'] = purch_analysis_age['Total Revenue (Normalized)'].map("${:.2f}".format)

# rearrange columns
purch_analysis_age = purch_analysis_age[[
    'Player Count', 'Total Purchases',
    'Avg Price', 'Total Revenue',
    'Total Revenue (Normalized)'
]]   

purch_analysis_age

,Player Count,Total Purchases,Avg Price,Total Revenue,Total Revenue (Normalized)
Age Groups,,,,,
<10,5,5,$2.76,$13.82,$2.76
10-14,3,3,$2.99,$8.96,$2.99
15-19,11,11,$2.76,$30.41,$2.76
20-24,34,36,$3.02,$108.89,$3.20
25-29,8,9,$2.90,$26.11,$3.26
30-34,6,7,$1.98,$13.89,$2.31
35-39,6,6,$3.56,$21.37,$3.56
40+,1,1,$4.65,$4.65,$4.65


In [14]:
# Top Spender Purchase Analysis
# group raw data by screen name
pymo_user = pymo_raw.groupby('SN')

# create DataFrame with count mean and sum of Price series by screen name
purch_analysis_user = pymo_user['Price'].agg(['count', 'mean', 'sum'])

# change DataFrame column names
purch_analysis_user = purch_analysis_user.rename(columns={
    'count': 'Total Purchases',
    'mean': 'Avg Price',
    'sum': 'Total Revenue'
})

# sort DataFrame values by Total Revenue in descending order 
purch_analysis_user = purch_analysis_user.sort_values(by='Total Revenue', ascending=False)

# reset index keeping old index (SN)
purch_analysis_user = purch_analysis_user.reset_index(drop=False)

# format columns with currecy values
purch_analysis_user['Avg Price'] = purch_analysis_user['Avg Price'].map("${:.2f}".format)
purch_analysis_user['Total Revenue'] = purch_analysis_user['Total Revenue'].map("${:.2f}".format)

# create new DataFrame with 5 top spenders
purch_analysis_top_spend = purch_analysis_user.iloc[:5]

# set SN as index
purch_analysis_top_spend.set_index('SN', inplace=True)

purch_analysis_top_spend

,Total Purchases,Avg Price,Total Revenue
SN,,,
Sundaky74,2,$3.71,$7.41
Aidaira26,2,$2.56,$5.13
Eusty71,1,$4.81,$4.81
Chanirra64,1,$4.78,$4.78
Alarap40,1,$4.71,$4.71


In [15]:
# Most Popular Item Purchase Analysis
# group raw data by Item Name
pymo_item = pymo_raw.groupby(['Item ID', 'Item Name', 'Price'])

# create DataFrame with count mean and sum of Price series by Item Name
purch_analysis_item = pymo_item['Price'].agg(['count', 'sum'])

# change DataFrame column names
purch_analysis_item = purch_analysis_item.rename(columns={
    'count': 'Total Purchases',
    'sum': 'Total Revenue'
})

# sort DataFrame values by Total Purchases in descending order
purch_analysis_most_pop = purch_analysis_item.sort_values(by='Total Purchases', ascending=False)

# create new DataFrame with grouped item data
purch_analysis_most_pop = purch_analysis_most_pop.reset_index(drop=False)

# format columns with currency values
purch_analysis_most_pop['Price'] = purch_analysis_most_pop['Price'].map('${:.2f}'.format)
purch_analysis_most_pop['Total Revenue'] = purch_analysis_most_pop['Total Revenue'].map('${:.2f}'.format)


# select rows of 5 most popular items based on purchase count
purch_analysis_most_pop = purch_analysis_most_pop.iloc[:5]

# set Item ID as index
purch_analysis_most_pop.set_index('Item ID', inplace=True)

purch_analysis_most_pop

,Item Name,Price,Total Purchases,Total Revenue
Item ID,,,,
94,Mourning Blade,$3.64,3,$10.92
90,Betrayer,$4.12,2,$8.24
111,Misery's End,$1.79,2,$3.58
64,Fusion Pummel,$2.42,2,$4.84
154,Feral Katana,$4.11,2,$8.22


In [16]:
# Most Profitable Item Purchase Analysis
# sort DataFrame by Total Revenue in descending order
purch_analysis_most_profit = purch_analysis_item.sort_values(by='Total Revenue', ascending=False)

# reset index keeping old index values
purch_analysis_most_profit = purch_analysis_most_profit.reset_index(drop=False)

# format columns with currency values
purch_analysis_most_profit['Price'] = purch_analysis_most_profit['Price'].map('${:.2f}'.format)
purch_analysis_most_profit['Total Revenue'] = purch_analysis_most_profit['Total Revenue'].map('${:.2f}'.format)


# select rows of 5 most profitable items
purch_analysis_most_profit = purch_analysis_most_profit.iloc[:5]

# set Item ID as index
purch_analysis_most_profit.set_index('Item ID', inplace=True)

purch_analysis_most_profit

,Item Name,Price,Total Purchases,Total Revenue
Item ID,,,,
94,Mourning Blade,$3.64,3,$10.92
117,"Heartstriker, Legacy of the Light",$4.71,2,$9.42
93,Apocalyptic Battlescythe,$4.49,2,$8.98
90,Betrayer,$4.12,2,$8.24
154,Feral Katana,$4.11,2,$8.22
